In [ ]:
from datetime import timedelta
import streamlit as st
import snowflake.snowpark.functions as F
import altair as alt

## Visualize Daily Sales on Snowsight

Before building our model, let's first visualize our data to get a feel for what our sales data looks like.

View the existing sales data

In [ ]:
SELECT *
    FROM tasty_byte_sales;

Find sales data specific to menu item `Lobster Mac & Cheese`

In [ ]:
tasty_byte_sales.to_df().where(F.col('menu_item_name').like('Lobster Mac & Cheese'))

### Plot the data
Let us visuualize the lobster sales data. We will use [Altair Charts](https://altair-viz.github.io/) with [Streamlit](https://streamlit.io).

In [ ]:
base = alt.Chart(lobster_sales.to_pandas())

total_sold = base.mark_line(point=True).encode(
    x=alt.X("DATE:O", timeUnit="monthdate", title="Date").scale(zero=False),
    y=alt.Y("TOTAL_SOLD", title="Total Sold").scale(zero=False),
)
st.altair_chart(total_sold,use_container_width=True)


## Build Forecast Model

### Dataset

Let us build the sales dataset for the city of `vancouver` with past **one** year's data. To able to refer to this data when building forecast model, we will persist the same in a table `vancouver_sales`.

In [ ]:
session = get_active_session()
tasty_bytes_df = tasty_byte_sales.to_df()

# Calculate 1 year interval since last sale date
date_range = tasty_bytes_df.select(F.max(F.col("date")).alias("max_date"))
max_d = date_range.first()[0]
# print(f"Current Date:{max_d} and its type {type(max_d)}")
interval = max_d - timedelta(365)
# print(f"Interval: {interval}")

van_sales = tasty_bytes_df.select(F.to_timestamp_ntz("date").alias("timestamp"),F.col("primary_city"),F.col("menu_item_name"),F.col("total_sold")).where(F.col("date") > interval)
# print(van_sales.queries)
van_sales.write.mode("overwrite").save_as_table("vancouver_sales")
van_sales

### Create Timeseries Data

The forecasting model relies on timeseries data. Let us create a view to build a timesries data from sales data `vancouver_sales` and filter it for `Lobster Mac & Cheese`.

In [ ]:
lobster_sales_view = "LOBSTER_SALES"
vancouver_sales_df = session.table("vancouver_sales")
lobster_sales_view_df = vancouver_sales_df.select(F.col("timestamp"),F.col("total_sold")).where(F.col("menu_item_name").like('Lobster Mac & Cheese'))
lobster_sales_view_df.createOrReplaceView(f"{lobster_sales_view}")

### Build a Forecast Model

In [ ]:
CREATE OR REPLACE forecast lobstermac_forecast (
    INPUT_DATA => TABLE({{ lobster_sales_view }}),
    TIMESTAMP_COLNAME => 'TIMESTAMP',
    TARGET_COLNAME => 'TOTAL_SOLD'
);

List all available forecast models.

In [ ]:
SHOW forecast;

### Forecast

Let us use the model to build a sales forecast for next 10 days. And persist the predictiosn on to a table named `macncheese_predictions`.

In [ ]:
CALL lobstermac_forecast!FORECAST(FORECASTING_PERIODS => 10)

In [ ]:
macncheese_predictions.to_df().write.mode("overwrite").save_as_table("macncheese_predictions")

### Visualize the Forecast

To visualize the Forecast i.e. plotting it as graph, we will beb building a multline chart. Let us `UNION` our actual sales data with the forecast(predictions) data.


In [ ]:
lobster_sales_view_df = session.table(lobster_sales_view)
sales_df = lobster_sales_view_df.with_column("forecast",F.lit(None)).where(F.col('timestamp') > '2023-03-01')
forecast_df = macncheese_predictions.to_df().select(F.col("TS").alias("timestamp"),F.col("forecast")).with_column("total_sold",F.lit(None))
sales_forecast_df = sales_df.union_all_by_name(forecast_df)
sales_forecast_df

In [ ]:
base = alt.Chart(sales_forecast_df.to_pandas())

# plot the actual sales
total_sold = base.mark_line(point=True, color="blue").encode(
    x=alt.X("TIMESTAMP:O",timeUnit="monthdate",title="Date").scale(zero=False),
    y=alt.Y("TOTAL_SOLD", title="Total Sold").scale(zero=False),
)

# plot the forecast
forecast = base.mark_line(point=True, color="orange").encode(
    x=alt.X("TIMESTAMP:O",timeUnit="monthdate",title="Date"),
    y=alt.Y(shorthand="FORECAST", title="Total Sold").scale(zero=False),
)
st.altair_chart(total_sold + forecast, use_container_width=True)

## Forecast Configuration

The `FORECAST` SQL funciton takes a `CONFIG_OBJECT` which we can use to tune various parameters. In the following example let us tune the `predection_interval`. Refer to [docs](https://docs.snowflake.com/sql-reference/classes/forecast/commands/create-forecast) for other configuration options.

In [ ]:
CALL lobstermac_forecast!FORECAST(FORECASTING_PERIODS => 10, CONFIG_OBJECT => {'prediction_interval': .5});